# AI Agents Crash Course

## Table of Contents

1. [Project Setup](#project-setup)
   - [Load Required Python Modules and Libraries](#load-required-python-modules-and-libraries)
   - [Load Environment Variables and Configure LLM](#load-environment-variables-and-configure-llm)
   - [Configure LLM](#configure-llm)

2. [Single Agent Single Tool Example](#single-agent-single-tool-example)
   - [Define Agents](#define-agents)
   - [Define a Writing Task](#define-a-writing-task)
   - [Create a Crew and Execute the Task](#create-a-crew-and-execute-the-task)
   - [Save the Response to a File](#save-the-response-to-a-file)

3. [File Reading Tools](#file-reading-tools)
   - [Define a file reader tool (OPENAI ONLY)](#define-a-file-reader-tool-openai-only---does-not-work-with-ollama-models)
   - [Troubleshooting Note](#troubleshooting-note)
   - [Define the Agent For Reading And Summarizing Files (OPENAI ONLY)](#define-the-agent-for-reading-and-summarizing-files-openai-only)
   - [Create Reading and Analyzing File Task (OPENAI ONLY)](#create-reading-and-analyzing-file-task-openai-only)
   - [Assemble A File-Processing Crew Workflow (OPENAI ONLY)](#assemble-a-file-processing-crew-workflow-openai-only)

4. [Building Multi-Agent Systems](#building-multi-agent-systems)
   - [Define our Serper Dev tool](#define-our-serper-dev-tool)
   - [Define the Internet Researcher Agent and Task](#define-the-internet-researcher-agent-and-task)
   - [Define The Summarization Agent](#define-the-summarization-agent)
   - [Define The Fact-Checking Agent](#define-the-fact-checking-agent)
   - [Create Multi-Agent Crew Workflow](#create-multi-agent-crew-workflow)
   - [Kickoff The Multi-Agent Crew Workflow](#kickoff-the-multi-agent-crew-workflow)

5. [Using YAML Based Agent and Workflow Definition](#using-yaml-based-agent-and-workflow-definition)
   - [Load YAML File](#load-yaml-file)
   - [Convert Agent Definitions to use YAML Config Details](#convert-agent-definitions-to-use-yaml-config-details)
   - [Create Multi-Agent Crew Workflow](#create-multi-agent-crew-workflow-1)

---

This notebook demonstrates various CrewAI features including:
- ✅ Single-agent workflows with basic tasks
- ✅ Multi-agent sequential workflows
- ✅ Tool integration (SerperDevTool for web search)
- ⚠️  File reading tools (OpenAI compatible only)
- ✅ YAML-based configuration
- ✅ LiteLLM debugging for troubleshooting
- ✅ Support for both OpenAI and Ollama LLMs

**Note on Ollama Compatibility**: File reading tools (FileReadTool) currently have compatibility issues with Ollama-based models due to a bug in the litellm/ollama integration. This notebook includes troubleshooting steps for this limitation.

---

REFERENCE:  https://www.dailydoseofds.com/ai-agents-crash-course-part-1-with-implementation/

# Project Setup

This notebook will guide you through setting up the project environment for using CrewAI. We will:

1. Install the required Python modules.
2. Set up a virtual environment.
3. Verify the installation.

In [ ]:
# Uncomment if you are not using devcontainers and want to set up a local environment
# 
# # Step 1: Create and activate a virtual environment
# #
# %python3 -m venv venv
# %source venv/bin/activate
# 
# # Step 2: Install required Python modules
# #
# %pip install -r requirements.txt
# 
# # Step 3: Verify installation
# #
# %pip list

# Load Required Python Modules and Libraries

In [ ]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown
from crewai import LLM, Agent, Task, Crew

In [ ]:
# LiteLLM Debug Control
# Run this cell to easily enable or disable litellm debugging
import litellm

def enable_litellm_debug():
    """Enable litellm debugging for detailed error diagnostics"""
    litellm._turn_on_debug()
    print("✅ LiteLLM debugging ENABLED")
    print("   - You will see detailed logs for LLM calls")
    print("   - Useful for troubleshooting connection and integration issues")

def disable_litellm_debug():
    """Disable litellm debugging to reduce verbose output"""
    litellm._turn_off_debug()
    print("🔇 LiteLLM debugging DISABLED")
    print("   - Reduced log output for cleaner execution")

def check_litellm_debug_status():
    """Check if litellm debugging is currently enabled"""
    # litellm doesn't have a direct way to check debug status,
    # so we'll check the logging level as a proxy
    import logging
    logger = logging.getLogger("litellm")
    if logger.level <= logging.DEBUG:
        print("🔍 LiteLLM debugging appears to be ENABLED")
    else:
        print("🔇 LiteLLM debugging appears to be DISABLED")

# Uncomment one of the following lines to control debugging:
# enable_litellm_debug()    # Enable debugging
# disable_litellm_debug()   # Disable debugging
check_litellm_debug_status()  # Check current status

In [ ]:
# Quick Debug Control - Run any of these cells for immediate control:

# 🟢 ENABLE DEBUGGING (run this cell)
# enable_litellm_debug()

# 🔴 DISABLE DEBUGGING (run this cell)  
# disable_litellm_debug()

# 🔍 CHECK STATUS (run this cell)
# check_litellm_debug_status()

print("💡 Tip: Uncomment and run the function you need above, or use the functions from the previous cell")

# Load Environment Variables and Configure LLM

This block loads environment variables from the `.env` file, including the OpenAI API Key, which is required to authenticate with OpenAI's services. It then configures the `LLM` object to use OpenAI's GPT-4 model. Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [ ]:
# Load environment variables from .env file
# Note: In devcontainer, variables are already loaded by dotenv feature,
# but load_dotenv() is safe and won't override existing environment variables
load_dotenv()

# Uncomment the code block below to use OpenAI with your API Key
# 
# api_key = os.getenv('OPENAI_API_KEY')
# if not api_key:
#     raise ValueError("OPENAI_API_KEY is not set in the .env file")

# Uncomment the code block below to use ollama
# 
OLLAMA_API_BASE = os.getenv('OLLAMA_API_BASE')

# Configure LLM

Configures the `LLM` object to use OpenAI's GPT-4 model. 

Alternatively, you can uncomment the provided code to configure the `LLM` object to use Ollama with a local model, provided Ollama is installed and running.

In [ ]:
# llm = LLM(
#     model="gpt-4o",  # Specify the OpenAI model you want to use
#     api_key=api_key
# )

# Uncomment the code block below to use Ollama with your local model
# Make sure to have Ollama installed and running
# 
llm = LLM(
    model="ollama/openhermes:latest",
    base_url=OLLAMA_API_BASE
)

In [ ]:
# Uncomment to test ollam connectivity
# 
import requests

# Debug: Check environment variables and test Ollama connection
# 
print("=== Environment Variables Debug ===")
print(f"OLLAMA_API_BASE: {OLLAMA_API_BASE}")
test_url = f"{OLLAMA_API_BASE}/api/tags"

# Test connection to your Ollama server
# 
try:
    response = requests.get(test_url, timeout=5)
    print(f"\n=== Ollama Connection Test ===")
    print(f"URL: {test_url}")
    print(f"Status: {response.status_code}")
    print(f"Response: {response.json()}")
except Exception as e:
    print(f"Connection failed: {e}")

# Debug: Check what base_url is actually being used
# 
print(f"\n=== CrewAI LLM Connection Test ===")
# print(f"ollama_base_url from env: {OLLAMA_API_BASE}")
print(f"LLM base_url: {llm.base_url}")
print(f"LLM model: {llm.model}")

# Test a simple LLM call
try:
    test_response = llm.call([{"role": "user", "content": "Say hello!"}])
    print("✅ LLM test successful!")
except Exception as e:
    print(f"❌ LLM test failed: {e}")

# Single Agent Single Tool Example

## Define Agents

This block defines multiple agents with specific roles, goals, and backstories. Each agent is configured to use the LLM defined earlier.

In [ ]:
senior_technical_writer = Agent(
    role="Senior Technical Writer",
    
    goal="""Craft clear, engaging, and well-structured
            technical content based on research findings""",
    
    backstory="""You are an experienced technical writer
                with expertise in simplifying complex
                concepts, structuring content for readability,
                and ensuring accuracy in documentation.""",
                
    llm=llm,
                
    verbose=True
)

research_analyst = Agent(
    role="Senior Research Analyst",
    goal="""Find, analyze, and summarize information 
            from various sources to support technical 
            and business-related inquiries.""",
    backstory="""You are a skilled research analyst with expertise 
                in gathering accurate data, identifying key trends, 
                and presenting insights in a structured manner.""",
    llm=llm,
    verbose=True
)

code_reviewer = Agent(
    role="Senior Code Reviewer",
    goal="""Review code for bugs, inefficiencies, and 
            security vulnerabilities while ensuring adherence 
            to best coding practices.""",
    backstory="""You are a seasoned software engineer with years of 
                experience in writing, reviewing, and optimizing 
                production-level code in multiple programming languages.""",
    llm=llm,
    verbose=True
)

legal_reviewer = Agent(
    role="Legal Document Expert Reviewer",
    goal="""Review contracts and legal documents to 
            ensure compliance with applicable laws and 
            highlight potential risks.""",
    backstory="""You are a legal expert with deep knowledge 
                of contract law, regulatory frameworks, 
                and risk mitigation strategies.""",
    llm=llm,
    verbose=True
)


## Define a Writing Task

This block defines a writing task for the Senior Technical Writer agent. The task includes a description, the agent responsible, and the expected output.

In [ ]:

writing_task = Task(
    description="""Write a well-structured, engaging,
                   and technically accurate article
                   on {topic}.""",
    
    agent=senior_technical_writer, 
    
    
    expected_output="""A polished, detailed, and easy-to-read
                       article on the given topic.""",
)


## Create a Crew and Execute the Task

This block creates a crew to manage the task and agents. It then kicks off the crew with the specified input and displays the response.

In [ ]:
crew = Crew(
    agents=[senior_technical_writer],
    tasks=[writing_task],
    verbose=True
)

response = crew.kickoff(inputs={"topic":"AI Agents"})

# Display the response in Markdown format
Markdown(response.raw)

## Save the Response to a File

This block saves the raw response from the crew execution to a Markdown file for further use or documentation.

In [ ]:
f = open("build/output.md", "w")
f.write(response.raw)
f.close()

# Define a file reader tool (OPENAI ONLY - DOES NOT WORK WITH OLLAMA MODELS)

In [ ]:
from crewai_tools import FileReadTool

file_read_tool = FileReadTool()

## Troubleshooting Note

**Issue:** The original code was encountering an `IndexError` in the litellm/ollama integration when using CrewAI tools. The error occurred in the prompt template processing where `messages[msg_i]` was accessing an index out of range.

**Root Cause:** This appears to be a compatibility issue between:
- CrewAI's tool integration system
- LiteLLM's Ollama prompt template processing
- The FileReadTool parameter validation

**Solution:** 
1. **Enabled debugging** with `litellm._turn_on_debug()` to get detailed error information
2. **Removed the FileReadTool** from the agent to avoid the tool integration bug
3. **Read file content directly** in Python and embedded it in the task description
4. **Simplified the workflow** to avoid dynamic parameter passing

This approach maintains the same functionality while working around the integration issue.

**Results:**
- This solution did not work either.  I was unable to get CrewAI FileReadTool or any custom specialized Tool Function to work with ollama based GenAI models.  The tool usage only worked for OpenAI for some reason.

## Define the Agent For Reading And Summarizing Files (OPENAI ONLY)

In [ ]:
from crewai import Agent

summarizer_agent = Agent(
    role="Senior Document Summarizer",

    goal="Extract and summarize key insights from provided files in 20 words or less.",
    backstory="""You are an expert in document analysis, skilled at extracting 
                 key details, summarizing content, and identifying critical 
                 insights from structured and unstructured text.""",
    llm=llm,
    tools=[file_read_tool],
    verbose=True
)

## Create Reading and Analyzing File Task (OPENAI ONLY)

In [ ]:
from crewai import Task

summarizer_task = Task(
    description=(
        "Use the FileReadTool to read the contents of {file_path}"
        "and provide a summary in 20 words or less. "
        "Ensure the summary captures the key insights "
        "and main points from the document."
    ),
    agent=summarizer_agent,
    tools=[file_read_tool],
    expected_output="A concise 20-word summary of the key points from the file.",
)

## Assemble A File-Processing Crew Workflow (OPENAI ONLY)

In [ ]:
from crewai import Crew

summarizer_crew = Crew(
    agents=[summarizer_agent],
    tasks=[summarizer_task],
    verbose=True
)

# Run the crew with the file input
result = summarizer_crew.kickoff(inputs={"file_path": "build/output.md"})

In [ ]:
# Display the response in Markdown format
from IPython.display import Markdown
Markdown(result.raw)

# Building multi-agent systems

## Define our Serper Dev tool

The Serper Dev Tool is a utility designed to interact with search engines and retrieve relevant information programmatically. It is particularly useful for tasks that require real-time data or insights from the web, such as:

1. Conducting research on specific topics.
2. Gathering up-to-date information for decision-making.
3. Enhancing the capabilities of agents by providing them with access to external data sources.

### How It Can Be Used
- **Integration with Agents**: The tool can be integrated into agents to enable them to fetch and process web-based information dynamically.
- **Custom Queries**: Developers can define specific queries to retrieve targeted information, making it adaptable to various use cases.
- **Real-Time Insights**: By leveraging the tool, agents can provide real-time insights and context, improving the quality and relevance of their outputs.

This tool is ideal for scenarios where static data is insufficient, and dynamic, real-time information is required to achieve the desired outcomes.

In [ ]:
from crewai_tools import SerperDevTool

serper_dev_tool = SerperDevTool()

## Define the Internet Researcher Agent and Task

This block defines an `Internet Researcher` agent and a corresponding research task. 
- The agent is equipped with the `SerperDevTool` to perform web-based research. 
- The agent's role is to find the most relevant and recent information about a given topic, leveraging its expertise in navigating the internet and gathering reliable data. 
- The task specifies the use of the `SerperDevTool` to extract key insights from multiple sources and produce a detailed research report with references.

In [ ]:
from crewai import Agent, Task

research_agent = Agent(
    role="Internet Researcher",
    goal="Find the most relevant and recent information about a given topic.",
    backstory="""You are a skilled researcher, adept at navigating the internet 
                 and gathering high-quality, reliable information.""",
    tools=[serper_dev_tool],
    verbose=True
)

research_task = Task(
    description="""Use the SerperDevTool to search for the 
                   most relevant and recent data about {topic}."""
                "Extract the key insights from multiple sources.",
    agent=research_agent,
    tools=[serper_dev_tool],
    expected_output="A detailed research report with key insights and source references."
)

## Define The Summarization Agent

This agent is responsible for condensing the research into a concise and structured summary. The Summarization Agent ensures that the research findings are structured, easy to read, and clear.

In [ ]:
summarizer_agent = Agent(
    role="Content Summarizer",
    goal="Condense the key insights from research into a short and informative summary.",
    backstory="""You are an expert in distilling complex information into concise, 
                 easy-to-read summaries.""",
    verbose=True
)

summarization_task = Task(
    description="Summarize the research report into a concise and informative paragraph. "
                "Ensure clarity, coherence, and completeness.",
    agent=summarizer_agent,
    expected_output="A well-structured summary with the most important insights."
)

## Define The Fact-Checking Agent

The Fact-Checking Agent will cross-check all summarized information with credible sources:

- The Fact-Checking Agent is responsible for validating the summarized information.
- The Serper Dev Tool is used again to cross-check facts with external sources.

In [ ]:
fact_checker_agent = Agent(
    role="Fact-Checking Specialist",
    goal="Verify the accuracy of information and remove any misleading or false claims.",
    backstory="""You are an investigative journalist with a knack for validating facts, 
                 ensuring that only accurate information is published.""",
    tools=[serper_dev_tool],
    verbose=True
)

fact_checking_task = Task(
    description="Verify the summarized information for accuracy using the SerperDevTool. "
                "Cross-check facts with reliable sources and correct any errors.",
    agent=fact_checker_agent,
    tools=[serper_dev_tool],
    expected_output="A fact-checked, verified summary of the research topic."
)

## Create Multi-Agent Crew Workflow

- All three agents are grouped into a Crew, each assigned their specific task.
- Tasks are executed sequentially in a structured workflow:  Research → Summarization → Fact-Checking.
- The topic is dynamically provided at runtime, making the workflow flexible for any research topic.

In [ ]:
from crewai import Crew, Process

research_crew = Crew(
    agents=[research_agent, summarizer_agent, fact_checker_agent],
    tasks=[research_task, summarization_task, fact_checking_task],
    process=Process.sequential,
    verbose=True
)


## Kickoff The Multi-Agent Crew Workflow

In [ ]:

result = research_crew.kickoff(inputs={"topic": "The impact of AI on job markets"})

# print("\nFinal Verified Summary:\n", result)
Markdown(result.raw)

# Using YAML Based Agent and Workflow Definition

## Load YAML File

In [ ]:
import yaml

with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
    
# Uncomment the following line to print the configuration
# 
# from pprint import pprint
# pprint(config)

## Convert Agent Definitions to use YAML Config Details

In [ ]:
research_agent = Agent(
    role=config["agents"]["research_agent"]["role"],
    goal=config["agents"]["research_agent"]["goal"],
    backstory=config["agents"]["research_agent"]["backstory"],
    tools=[serper_dev_tool],
    verbose=True
)

research_task = Task(
    description=config["tasks"]["research_task"]["description"],
    agent=research_agent,
    tools=[serper_dev_tool],
    expected_output=config["tasks"]["research_task"]["expected_output"]
)

summarization_agent = Agent(
    role=config["agents"]["summarization_agent"]["role"],
    goal=config["agents"]["summarization_agent"]["goal"],
    backstory=config["agents"]["summarization_agent"]["backstory"],
    verbose=True
)

fact_checker_agent = Agent(
    role=config["agents"]["fact_checker_agent"]["role"],
    goal=config["agents"]["fact_checker_agent"]["goal"],
    backstory=config["agents"]["fact_checker_agent"]["backstory"],
    tools=[serper_dev_tool],
    verbose=True
)

summarization_task = Task(
    description=config["tasks"]["summarization_task"]["description"],
    agent=summarization_agent,
    expected_output=config["tasks"]["summarization_task"]["expected_output"],
)

fact_checking_task = Task(
    description=config["tasks"]["fact_checking_task"]["description"],
    agent=fact_checker_agent,
    tools=[serper_dev_tool],
    expected_output=config["tasks"]["fact_checking_task"]["expected_output"],
)


## Create Multi-Agent Crew Workflow

- All three agents are grouped into a Crew, each assigned their specific task.
- Tasks are executed sequentially in a structured workflow:  Research → Summarization → Fact-Checking.
- The topic is dynamically provided at runtime, making the workflow flexible for any research topic.

In [ ]:
from crewai import Crew, Process

research_crew = Crew(
    agents=[research_agent, summarizer_agent, fact_checker_agent],
    tasks=[research_task, summarization_task, fact_checking_task],
    process=Process.sequential,
    verbose=True
)

result = research_crew.kickoff(inputs={"topic": "The impact of AI on job markets"})
print("\nFinal Verified Summary:\n", result)